<a href="https://colab.research.google.com/github/mhkarimid/Data/blob/main/Data-porteioning-exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.5"
export JULIA_PKG_PRECOMPILE_AUTO=0
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  echo "Installing PyCall.jl..."
  julia -e 'using Pkg; Pkg.add("PyCall"); Pkg.build("PyCall")'
  julia -e 'println("Success")'

fi

Installing Julia 1.8.5 on the current Colab Runtime...
2023-03-18 01:25:39 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.5-linux-x86_64.tar.gz [130873886/130873886] -> "/tmp/julia.tar.gz" [1]
Installing PyCall.jl...
  Installing known registries into `~/.julia`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Preferences ───── v1.3.0
   Installed VersionParsing ── v1.3.0
   Installed Conda ─────────── v1.8.0
   Installed JSON ──────────── v0.21.3
   Installed MacroTools ────── v0.5.10
   Installed Parsers ───────── v2.5.8
   Installed PyCall ────────── v1.95.1
   Installed SnoopPrecompile ─ v1.0.3
    Updating `~/.julia/environments/v1.8/Project.toml`
  [438e738f] + PyCall v1.95.1
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [8f4d0f93] + Conda v1.8.0
  [682c06a0] + JSON v0.21.3
  [1914dd2f] + MacroTools v0.5.10
  [69de0a69] + Parsers v2.5.8
  [21216c6a] + Preferences v1.3.0
  [438e738f] + 

In [3]:
%pip install -Uq pysr pytorch_lightning
from julia import Julia

julia = Julia(compiled_modules=False, threads='auto')
from julia import Main
from julia.tools import redirect_output_streams

redirect_output_streams()
import pysr

# We don't precompile in colab because compiled modules are incompatible static Python libraries:
pysr.install(precompile=False)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 10.8 MB/s eta 0:00:00
Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
      Ubuntu 20.04.5 LTS
  uname: Linux 5.10.147+ #1 SMP Sat Dec 10 16:00:40 UTC 2022 x86_64 x86_64
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz: 
              speed         user         nice          sys         idle          irq
       #1  2200 MHz    

[ Info: Julia version info
[ Info: Julia executable: /usr/local/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /usr/bin/python3
│     libpython: /usr/lib/x86_64-linux-gnu/libpython3.9.so.1.0
│ Python:
│     python: /usr/bin/python3
└     libpython: 
     Cloning git-repo `https://github.com/MilesCranmer/SymbolicRegression.jl`
     Cloning git-repo `https://github.com/JuliaParallel/ClusterManagers.jl`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed RealDot ────────────────────────── v0.1.0
   Installed IRTools ────────────────────────── v0.4.8
   Installed BitTwiddlingConvenienceFunctions ─ v0.1.5
   Installed DynamicPolynomials ─────────────── v0.4.6
   Installed GPUArraysCore ──────────────────── v0.1.4
   Installed SIMDTypes ──────────────────────── v0.1.0
   Installed IrrationalConstants ────────────── v0.2.2
   Installed AutoH

In [4]:
import sympy
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pysr import PySRRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score

In [5]:
df=pd.read_csv('https://raw.githubusercontent.com/mhkarimid/Data/main/newdata.csv', 
               names=['Time (min)', 'Gold grade (g/ton)', 'Pyrite content (wt.%)',
               'Ore content (wt.%)', 'pH', 'ORP (mV)', 'D-Gal (wt.%)',
               'D-Suc (wt.%)', 'D-FrU (wt.%)', 'Ferric (g/lit)', 'Pyrite dissolution (%)'])
df.head()

,Time (min),Gold grade (g/ton),Pyrite content (wt.%),Ore content (wt.%),pH,ORP (mV),D-Gal (wt.%),D-Suc (wt.%),D-FrU (wt.%),Ferric (g/lit),Pyrite dissolution (%)
0,-0.500000,0.5,0.5,-0.444444,-0.191734,-0.460602,-0.155844,-0.155844,-0.155844,-0.177262,-0.339150
1,-0.456522,0.5,0.5,-0.444444,-0.201303,-0.443605,-0.155844,-0.155844,-0.155844,-0.161934,-0.323117
2,-0.413043,0.5,0.5,-0.444444,-0.244366,-0.432274,-0.155844,-0.155844,-0.155844,-0.165186,-0.327591
3,-0.369565,0.5,0.5,-0.444444,-0.273074,-0.304795,-0.155844,-0.155844,-0.155844,-0.131744,-0.278188
4,-0.326087,0.5,0.5,-0.444444,-0.225227,-0.214143,-0.155844,-0.155844,-0.155844,-0.118739,-0.251342


In [6]:

gg=np.sqrt(df['Time (min)']**2+
            df['Gold grade (g/ton)']**2+
            df['Pyrite content (wt.%)']**2+
            df['Ore content (wt.%)']**2+
            df['pH']**2+\
            df['ORP (mV)']**2+
            df['D-Gal (wt.%)']**2+
            df['D-Suc (wt.%)']**2+
            df['D-FrU (wt.%)']**2)
    
df['Time (min)']=df['Time (min)']/gg
df['Gold grade (g/ton)']=df['Gold grade (g/ton)']/gg
df['Pyrite content (wt.%)']=df['Pyrite content (wt.%)']/gg
df['Ore content (wt.%)']=df['Ore content (wt.%)']/gg
df['pH']=df['pH']/gg
df['ORP (mV)']=df['ORP (mV)']/gg
df['D-Gal (wt.%)']=df['D-Gal (wt.%)']/gg
df['D-Suc (wt.%)']=df['D-Suc (wt.%)']/gg
df['D-FrU (wt.%)']=df['D-FrU (wt.%)']/gg

def sig(x):
 return 1/(1 + np.exp(-x))
 

df=sig(df)
df.head()

,Time (min),Gold grade (g/ton),Pyrite content (wt.%),Ore content (wt.%),pH,ORP (mV),D-Gal (wt.%),D-Suc (wt.%),D-FrU (wt.%),Ferric (g/lit),Pyrite dissolution (%)
0,0.390836,0.609164,0.609164,0.402637,0.457557,0.399193,0.465473,0.465473,0.465473,0.455800,0.416016
1,0.397959,0.611448,0.611448,0.400586,0.454491,0.400769,0.464729,0.464729,0.464729,0.459605,0.419916
2,0.406363,0.612734,0.612734,0.399431,0.444174,0.402113,0.464309,0.464309,0.464309,0.458797,0.418827
3,0.411799,0.618307,0.618307,0.394420,0.434517,0.427013,0.462484,0.462484,0.462484,0.467112,0.430898
4,0.418112,0.624058,0.624058,0.389243,0.443173,0.445947,0.460590,0.460590,0.460590,0.470350,0.437493


In [7]:
XX=df.drop(['Gold grade (g/ton)','Ferric (g/lit)','Pyrite dissolution (%)'], axis=1)
yy=df['Pyrite dissolution (%)']

Rtr=[]
RMSEtr=[]

Rte=[]
RMSEte=[]

for i in range(1,19):
    X_tr, X_te, y_tr, y_te = train_test_split(XX, yy, test_size=0.05*i)
    X = X_tr.values.tolist()
    y = y_tr.transpose().values.tolist()
    Xte = X_te.values.tolist()
    yte = y_te.transpose().values.tolist()
    default_pysr_params = dict(
        populations=10,
        model_selection="accuracy",
        loss="L1DistLoss()",
        maxsize=30,
        precision=16,
    )
    model = PySRRegressor( 
        niterations=30,
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["exp","sinh", "cosh"],
        **default_pysr_params
    )
    model.fit(X, y)
    y2=model.predict(Xte)
    
    dk=pd.DataFrame(data=[yte,y2])
    d=dk.transpose().corr()[1]
    SS=d.squeeze()[0]
    Rte.append(SS)
    rmse=(mean_squared_error(yte, y2))
    RMSEte.append(rmse)  
    
    y3=model.predict(X)
    dk=pd.DataFrame(data=[y,y3])
    d=dk.transpose().corr()[1]
    SS=d.squeeze()[0]
    Rtr.append(SS)
    rmse=(mean_squared_error(y, y3))
    RMSEtr.append(rmse)
    
    


/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 1.800e+03
Head worker occupation: 0.7%
Progress: 6 / 300 total iterations (2.000%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.518e-02  7.388e-01  x4
7           2.483e-02  2.377e-03  (x4 - ((x4 * x3) * 0.02203))
11          2.477e-02  6.111e-04  (x4 - (((x4 * x3) / (x7 - -0.05353)) * 0.00968))
18          2.449e-02  1.613e-03  sinh(exp((exp(x4 * x6) * (sinh(x4) * x0)) * x4) / exp(1.062 / 1.155))
19          2.283e-02  7.028e-02  sinh(exp((sinh(x4) * sinh(x4)) * (exp(x4 * x6) * x4)) / exp(1.062 / 1.155))
20          2.278e-02  1.955e-03  sinh(exp((sinh(x4) * sinh(x4)) * (exp(sinh(x4 * x6)) * x4)) / exp(1.062 / 1.155))
24          2.266e-02  1.346e-03  sinh(exp((sinh(x4) * sinh(x4)) * (exp(((cosh(x4) / 1.155) * x4) * x6) * x4)) / exp(1.062 / 1.155))
27          2.261e-02  6.517e-04  sinh(exp((sinh(x4) * sinh(x4)) * (exp((((cosh(x4) / 0.7695) * x4) / sinh(1.155)) * x6) * x4)) / exp(1.062 / 1.

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 5.850e+02
Head worker occupation: NaN%
Progress: 2 / 300 total iterations (0.667%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.533e-02  7.287e-01  x4
3           2.505e-02  5.400e-03  (x4 * 0.9893)
7           2.478e-02  2.700e-03  (x4 - ((0.02434 * x4) * x4))
8           2.477e-02  4.884e-04  (x4 - ((0.02434 * sinh(x4)) * x4))
11          2.393e-02  1.142e-02  (x4 - (((x4 * (x4 - x3)) * x2) * x6))
14          2.341e-02  7.240e-03  (x4 - (((x3 * ((exp(0.02434) * x4) - x3)) * x2) * x6))

Press 'q' and then <enter> to stop execution early.

Cycles per second: 1.720e+03
Head worker occupation: 0.2%
Progress: 11 / 300 total iterations (3.667%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.533e-02  7.287e-01  x4
3           2.505e-02  5.400e-03  (x4 * 0.9893)
6           2.452e-02  7.240e-03  (x4 / cosh(x4 * -0.3645))
7           

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


...

Started!

Cycles per second: 5.870e+03
Head worker occupation: 1.2%
Progress: 17 / 300 total iterations (5.667%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.632e-02  6.989e-01  x4
3           2.626e-02  1.222e-03  (x4 * 0.997)
7           2.499e-02  1.239e-02  (cosh(sinh(0.619 - x0)) * x4)
11          2.463e-02  3.689e-03  (cosh(-1.468 / (exp(cosh(exp(x4))) * x0)) * x4)
12          2.458e-02  1.955e-03  (cosh((1.458 / ((x4 - x7) - -2.283)) - x4) * x4)
13          2.406e-02  2.122e-02  (cosh(-0.3958 / ((exp(cosh(exp(x4))) * x0) * x4)) * x4)
15          2.150e-02  5.627e-02  (cosh(x5 / (((exp(sinh(exp(x4))) * x5) - 1.276) + -1.191)) * x4)
19          2.119e-02  3.565e-03  ((cosh(-0.3958 / (((exp(cosh(exp(x4))) * x4) - exp(x4)) * sinh(x0))) / 1.004) * x4)
20          2.110e-02  4.404e-03  ((cosh(-0.3958 / (((exp(cosh(exp(x4))) * x4) - exp(x4)) * sinh(sinh(x0)))) / 1.004) * x4)
22          2.090e-02  4.660e-03  ((cosh(-0.3

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 1.080e+04
Head worker occupation: 1.2%
Progress: 30 / 300 total iterations (10.000%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.631e-02  7.064e-01  x4
3           2.618e-02  2.447e-03  (x4 * 0.9937)
6           2.544e-02  9.576e-03  (x4 / cosh(x0 - x1))
7           2.502e-02  1.624e-02  (x4 / cosh(exp(x4) - 1.527))
8           2.423e-02  3.225e-02  (x4 / cosh(x0 - (x1 * 0.8184)))
9           2.396e-02  1.129e-02  (x4 / cosh(x6 - sinh(sinh(sinh(x4)))))
10          2.287e-02  4.647e-02  (x4 * exp((x6 - x4) / sinh(exp(x6))))
11          2.264e-02  9.814e-03  (x4 * exp((x6 - x4) / sinh(exp(sinh(x6)))))
12          2.237e-02  1.228e-02  (x4 * exp((x6 - x4) / sinh(exp(sinh(sinh(x6))))))
14          2.199e-02  8.619e-03  (x4 * exp((x6 - x4) / sinh(sinh(exp(sinh(x6) - 0.1718)))))
15          2.097e-02  4.750e-02  (x4 * exp((x6 - x4) / sinh(sinh((x6 * exp(x6)) + x0))))
16          2.094e-02  1.466e

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 1.600e+04
Head worker occupation: 1.9%
Progress: 46 / 300 total iterations (15.333%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.515e-02  7.260e-01  x4
3           2.460e-02  1.111e-02  (x4 * 0.991)
5           2.458e-02  2.442e-04  ((x4 * 0.538) / 0.5435)
6           2.296e-02  6.819e-02  ((x4 / cosh(0.428)) + 0.0342)
8           2.289e-02  1.712e-03  sinh(x4 / cosh(sinh(x4) - 0.2109))
10          2.235e-02  1.186e-02  sinh(x4 / cosh(cosh(x2) * (x4 - 0.2109)))
13          2.187e-02  7.406e-03  ((cosh(exp((0.5645 / x4) + -0.3723) * -0.1654) * x4) / 1.059)
14          1.964e-02  1.076e-01  ((cosh((cosh(sinh(0.681) / x4) - x4) * -0.1654) * x4) / 1.059)
15          1.962e-02  9.770e-04  ((cosh((cosh(sinh(0.681) / x4) - x4) * sinh(-0.1654)) * x4) / 1.059)
16          1.923e-02  2.022e-02  ((cosh((cosh(sinh(-0.725) / x4) - (x4 + x7)) * -0.1654) * x4) / 1.059)
19          1.889e-02  5.911e-03  ((

In [8]:
Results=pd.DataFrame(list(zip(RMSEtr, Rtr, RMSEte, Rte)), columns=['RMSE (training)', 
                                                               'Correlation factor (training)',
                                                              'RMSE (testing)', 
                                                               'Correlation factor (testing)'])

In [9]:
Results

,RMSE (training),Correlation factor (training),RMSE (testing),Correlation factor (testing)
0,0.000491,0.929603,0.000534,0.921149
1,0.000403,0.941890,0.000432,0.940572
2,0.000437,0.937308,0.000446,0.932700
3,0.000439,0.936744,0.000433,0.936749
4,0.000510,0.926749,0.000484,0.928408
5,0.000495,0.931612,0.000533,0.923495
6,0.000482,0.931744,0.000491,0.932869
7,0.000403,0.940640,0.000433,0.940472
8,0.000406,0.942780,0.000427,0.937425
9,0.000508,0.928134,0.000481,0.934123


In [10]:
Results.to_excel("SRGAResutls.xlsx") 

In [ ]:
XX=df.drop(['Gold grade (g/ton)','Ferric (g/lit)','Pyrite dissolution (%)'], axis=1)
yy=df['Pyrite dissolution (%)']

Rtr=[]
RMSEtr=[]

Rte=[]
RMSEte=[]


for i in range(1,10):
  X_tr, X_te, y_tr, y_te = train_test_split(XX, yy, test_size=0.05*i, random_state=42)
  X = X_tr.values.tolist()
  y = y_tr.transpose().values.tolist()

  Xte = X_te.values.tolist()
  yte = y_te.transpose().values.tolist()
  default_pysr_params = dict(
      populations=10,
      model_selection="accuracy",
      loss="L1DistLoss()",
      maxsize=30,
      precision=16,
      )
  model = PySRRegressor( 
      niterations=100,
      binary_operators=["+", "-", "*", "/"],
      unary_operators=["inv(x) = 1/x", "P2(x)=x", "P4(x)=x", "exp", "neg","sinh", "cosh"],
      extra_sympy_mappings={"inv": lambda x: 1/x, "P2": lambda x: x**0.3, "P4": lambda x: x**0.7},
      **default_pysr_params
      )
  model.fit(X, y)
  y2=model.predict(Xte)
  
  dk=pd.DataFrame(data=[yte,y2])
  d=dk.transpose().corr()[1]
  SS=d.squeeze()[0]
  Rte.append(SS)
  rmse=(mean_squared_error(yte, y2))
  RMSEte.append(rmse)
  
  y3=model.predict(X)
  
  dk=pd.DataFrame(data=[y,y3])
  d=dk.transpose().corr()[1]
  SS=d.squeeze()[0]
  Rtr.append(SS)
  rmse=(mean_squared_error(y, y3))
  RMSEtr.append(rmse)

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 1.410e+03
Head worker occupation: 0.1%
Progress: 4 / 1000 total iterations (0.400%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.258e-02  9.436e-01  x4
3           7.587e-02  4.253e-02  (x4 * 0.8384)
11          6.909e-02  1.170e-02  (inv((-1.51 / P4(0.04916 - x4)) + -0.669) * 1.296)
12          6.775e-02  1.972e-02  (P2(inv(P2(-1.51 / (0.03696 - x4)) + -0.669)) * 1.296)

Press 'q' and then <enter> to stop execution early.

Cycles per second: 2.500e+03
Head worker occupation: 0.2%
Progress: 14 / 1000 total iterations (1.400%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.258e-02  9.436e-01  x4
3           7.587e-02  4.253e-02  (x4 * 0.8384)
6           7.361e-02  1.008e-02  ((x4 / 1.057) / cosh(x4))
8           6.287e-02  7.891e-02  (((x4 * x4) - sinh(0.05112)) + x4)
14          6.262e-02  6.523e-04  (inv(P2(P2(-1.51) / (0.073

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 1.610e+03
Head worker occupation: 0.1%
Progress: 5 / 1000 total iterations (0.500%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.246e-02  9.427e-01  x4
3           7.587e-02  4.173e-02  (x4 * 0.8306)
6           7.581e-02  3.257e-04  (P2(x4 + -0.002268) * 0.84)
7           7.574e-02  9.770e-04  (P2(P2(x4 + -0.0045)) * 0.827)
9           7.556e-02  1.222e-03  (P2(P4((x5 * -0.03705) + x4)) * 0.846)
14          7.507e-02  1.274e-03  (P2((sinh(neg(P4(0.00934))) * (neg(-0.2429) + x2)) + x4) * 0.84)

Press 'q' and then <enter> to stop execution early.

Cycles per second: 2.890e+03
Head worker occupation: 0.2%
Progress: 18 / 1000 total iterations (1.800%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.246e-02  9.427e-01  x4
3           7.587e-02  4.173e-02  (x4 * 0.8306)
6           7.581e-02  3.257e-04  (P2(x4 + -0.002268) * 0.84)
7 

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 4.180e+03
Head worker occupation: 0.6%
Progress: 13 / 1000 total iterations (1.300%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           8.234e-02  9.413e-01  x4
3           7.568e-02  4.200e-02  (x4 * 0.8345)
5           7.556e-02  7.330e-04  (x4 / cosh(sinh(x0)))
6           6.830e-02  1.011e-01  (x4 / cosh(x4 + x4))
7           6.732e-02  1.426e-02  (sinh(x4) / cosh(x4 + x4))
8           6.696e-02  5.386e-03  sinh(sinh(x4 / cosh(x4 + x4)))
9           6.580e-02  1.724e-02  (sinh(x4 / cosh(x4 + x4)) + -0.01477)
22          6.549e-02  3.388e-04  P4(x4 * sinh(sinh(inv(sinh(P4(cosh(P4(x4 * P4(inv(cosh(x4 + 0.746)))) * neg(sinh(1.252)))))))))

Press 'q' and then <enter> to stop execution early.

Cycles per second: 3.440e+03
Head worker occupation: 0.3%
Progress: 22 / 1000 total iterations (2.200%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1       

<lambdifygenerated-9>:2: RuntimeWarning: invalid value encountered in power
  return (0.582750582750583*x1**0.7*sinh(0.91324200913242*sinh(sinh((x1 - x4**0.3)*(sinh(x1) - 0.537))) - 0.0656) + 0.582750582750583*x4*(x4 + 1.716))


ValueError: ignored

In [ ]:

i

3

In [ ]:
y2

array([       nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan, 0.63350959,        nan,
              nan,        nan,        nan,        nan,        nan,
       0.60340461,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan, 0.59221324,
              nan, 0.72008743,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan, 0.65934818,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan, 0.65031747,
              nan,        nan,        nan, 0.70733686,        nan,
              nan, 0.57111239,        nan, 0.75134714,        nan,
       0.73789498,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        

In [ ]:
Xte

[[0.152173913,
  -0.5,
  0.555555556,
  0.186256361,
  0.134298605,
  -0.155844156,
  -0.155844156,
  0.844155844],
 [-0.02173913,
  0.5,
  -0.111111111,
  -0.019485266,
  0.292938831,
  -0.155844156,
  -0.064935065,
  -0.155844156],
 [-0.282608696,
  -0.1,
  -0.111111111,
  0.119270715,
  -0.262301962,
  -0.064935065,
  -0.155844156,
  -0.155844156],
 [-0.456521739,
  -0.1,
  -0.444444444,
  0.109701337,
  -0.437939356,
  -0.155844156,
  0.844155844,
  -0.155844156],
 [-0.065217391,
  0.1,
  -0.444444444,
  -0.057762778,
  0.080474242,
  -0.155844156,
  -0.155844156,
  -0.155844156],
 [0.065217391,
  0.1,
  -0.111111111,
  0.195825739,
  0.24194733,
  -0.064935065,
  -0.155844156,
  -0.155844156],
 [-0.326086957,
  0.5,
  -0.444444444,
  -0.119963735,
  -0.13765607,
  0.844155844,
  -0.155844156,
  -0.155844156],
 [-0.02173913,
  -0.3,
  -0.444444444,
  0.133624782,
  0.216451579,
  -0.155844156,
  -0.155844156,
  0.844155844],
 [0.152173913,
  0.1,
  0.555555556,
  0.033146313,
  0.1